In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction'

In [4]:
from mlproject.constants import *

[2025-04-20 20:31:45,181 : INFO : __init__ : Logger has been set up successfully!]


In [5]:
from dataclasses import dataclass
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
import json
import mlflow
import dagshub
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging

In [6]:
from dataclasses import dataclass
import pandas as pd
import joblib
import json
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json
from mlproject import logger

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    metric_file_path: Path
    preprocessor_path: Path
    test_raw_data: Path  
    target_column: str
    all_params: dict

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LGBMClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            test_raw_data=config.test_raw_data,  
            target_column=schema.name,
            all_params=params
        )
        return model_evaluation_config

In [ ]:
import os
import json
import joblib
import mlflow
import dagshub
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlflow.models.signature import infer_signature
from mlproject import logger  
from mlproject.entities.config_entity import ModelEvaluationConfig  

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        # Setup DagsHub tracking
        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "39af2ec9240d8439ca7a568d37e4c8566f0e4507"

        dagshub.init(
            repo_owner="JavithNaseem-J",
            repo_name="Telecom-Customer-Churn-Prediction"
        )
        mlflow.set_tracking_uri(
            "https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow"
        )
        mlflow.set_experiment("Telecom-Customer-Churn-Prediction")

    def evaluate(self):
        try:
            mlflow.lightgbm.autolog()

            if not Path(self.config.test_raw_data).exists():
                raise FileNotFoundError(f"Test data not found at {self.config.test_raw_data}")
            if not Path(self.config.preprocessor_path).exists():
                raise FileNotFoundError(f"Preprocessor not found at {self.config.preprocessor_path}")
            if not Path(self.config.model_path).exists():
                raise FileNotFoundError(f"Model not found at {self.config.model_path}")

            with mlflow.start_run():
                # Set run tags
                mlflow.set_tag("model_type", "CatBoostClassifier")
                mlflow.set_tag("evaluation_stage", "testing")

                # Load preprocessor and model
                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)

                # NOTE: Skip this if autolog is capturing params
                # mlflow.log_params(self.config.all_params)

                # Load and prepare test data
                logger.info(f"Loading test data from {self.config.test_raw_data}...")
                test_data = pd.read_csv(self.config.test_raw_data)
                target_column = self.config.target_column

                if target_column not in test_data.columns:
                    raise KeyError(f"Target column '{target_column}' not found in test data.")

                test_y = test_data[target_column]
                test_x = test_data.drop(columns=[target_column])
                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                # Preprocess and predict
                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                logger.info("Making predictions...")
                predictions = model.predict(test_x_transformed)

                # Evaluation metrics
                logger.info("Evaluating model performance...")
                try:
                    precision = precision_score(test_y, predictions, average="weighted")
                    recall = recall_score(test_y, predictions, average="weighted")
                    f1 = f1_score(test_y, predictions, average="weighted")
                except Exception as metric_error:
                    logger.warning(f"Metric error: {str(metric_error)}. Using zero_division=0 fallback.")
                    precision = precision_score(test_y, predictions, average="weighted", zero_division=0)
                    recall = recall_score(test_y, predictions, average="weighted", zero_division=0)
                    f1 = f1_score(test_y, predictions, average="weighted", zero_division=0)

                metrics = {
                    "accuracy": accuracy_score(test_y, predictions),
                    "precision": precision,
                    "recall": recall,
                    "f1": f1
                }

                # Log metrics manually (still needed)
                mlflow.log_metrics(metrics)

                # Log model (with signature and registry name)
                signature = infer_signature(test_x_transformed, predictions)
                mlflow.sklearn.log_model(
                    model,
                    artifact_path="TelecomCustomerChurnModel",
                    signature=signature,
                    registered_model_name="TelecomCustomerChurnModel"
                )

                # Print and save metrics
                logger.info(f"Evaluation Metrics:\n{json.dumps(metrics, indent=2)}")
                metrics_file = Path(self.config.root_dir) / "metrics.json"
                with open(metrics_file, "w") as f:
                    json.dump(metrics, f, indent=4)
                logger.info(f"Metrics saved to {metrics_file}")

                return metrics

        except Exception as e:
            logger.error(f"Error during model evaluation: {str(e)}")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-04-20 20:31:52,140 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-04-20 20:31:52,144 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-04-20 20:31:52,160 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-04-20 20:31:52,167 : INFO : common : created directory at: artifacts]
[2025-04-20 20:31:52,171 : INFO : common : created directory at: artifacts/model_evaluation]


Accessing as JavithNaseem-J

Initialized MLflow to track repo "JavithNaseem-J/Telecom-Customer-Churn-Prediction"

Repository JavithNaseem-J/Telecom-Customer-Churn-Prediction initialized!

[2025-04-20 20:31:56,993 : INFO : 527151452 : Loading preprocessor and model...]
[2025-04-20 20:31:57,154 : INFO : 527151452 : Loading test data from artifacts/data_transformation/test.csv...]
[2025-04-20 20:31:57,182 : INFO : 527151452 : Test data shape: X=(2587, 19), y=(2587,)]
[2025-04-20 20:31:57,182 : INFO : 527151452 : Preprocessing test features...]
[2025-04-20 20:31:57,191 : INFO : 527151452 : Making predictions...]
[2025-04-20 20:31:57,367 : INFO : 527151452 : Evaluating model performance...]


f:\ProgramFiles\anaconda3\envs\tele-comm\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Registered model 'TelecomCustomerChurnModel' already exists. Creating a new version of this model...
2025/04/20 20:32:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: TelecomCustomerChurnModel, version 11


[2025-04-20 20:32:20,151 : INFO : 527151452 : Evaluation Metrics:
{
  "accuracy": 0.847313490529571,
  "precision": 0.8474327707126296,
  "recall": 0.847313490529571,
  "f1": 0.8473061440986339
}]
[2025-04-20 20:32:20,159 : INFO : 527151452 : Metrics saved to artifacts\model_evaluation\metrics.json]


Created version '11' of model 'TelecomCustomerChurnModel'.


🏃 View run gaudy-horse-658 at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/1/runs/3c1ad698a5754dfba49ac19b48d52421
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/1
